# Monte Carlo Poker Calculator ♠️♥️♣️♦️

**Интерактивный калькулятор для Покера с использованием метода Монте-Карло**

Позволяет рассчитать вероятность победы в Texas Hold'em, симулируя тысячи раздач с известными картами.


## Как это работает:

1. **Запустите все ячейки кода** (Kernel → Restart Kernel and Run All Cells... → Restart)
2. **Перейдите вниз, к активному виджету** (ячейка 5)
3. **Выберите карты игрока** (2 карты)
4. **Выберите карты на столе** (флоп, терн, ривер)
5. **Укажите количество активных игроков** (2-9)
6. **Установите количество симуляций** (до 100.000)
7. **Нажмите "🎲 Рассчитать"** 

---


In [ ]:
# Импорты и основная функция расчёта методом Монте-Карло

import random
from typing import List, Tuple
from treys import Evaluator, Deck, Card

# Виджеты для интерактивности
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Визуализация
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge

# Глобальный evaluator для переиспользования
EVALUATOR = Evaluator()


def monte_carlo(hero_cards: List[int],
                board_cards: List[int],
                active: int,
                n_simulations: int) -> Tuple[int, int, int]:
    """
    Симуляция раздач методом Монте-Карло для расчёта шансов.

    Args:
        hero_cards: Карты героя (2 карты в формате treys)
        board_cards: Карты на столе (от 0 до 5 карт)
        active: Количество активных игроков (включая героя)
        n_simulations: Количество симуляций

    Returns:
        Tuple из (wins, ties, losses)
    """
    # Создаём колоду и убираем известные карты
    deck = Deck()
    used_cards = set(hero_cards + board_cards)
    remaining_cards = [c for c in deck.cards if c not in used_cards]

    wins = ties = losses = 0

    # Цикл симуляций
    for _ in range(n_simulations):
        # Перемешиваем оставшиеся карты
        random.shuffle(remaining_cards)

        # Раздаём карты противникам
        card_index = 0
        villains = []
        for _ in range(active - 1):
            villain = remaining_cards[card_index:card_index + 2]
            villains.append(villain)
            card_index += 2

        # Добираем доску до 5 карт
        sim_board = board_cards[:]
        cards_needed = 5 - len(sim_board)
        if cards_needed > 0:
            sim_board.extend(remaining_cards[card_index:card_index + cards_needed])

        # Оценка рук (в treys меньше = лучше)
        hero_score = EVALUATOR.evaluate(sim_board, hero_cards)

        # Находим лучшего противника
        best_villain_score = float('inf')
        for villain in villains:
            villain_score = EVALUATOR.evaluate(sim_board, villain)
            if villain_score < best_villain_score:
                best_villain_score = villain_score

        # Подсчёт результатов
        if hero_score < best_villain_score:
            wins += 1
        elif hero_score == best_villain_score:
            ties += 1
        else:
            losses += 1

    return wins, ties, losses

In [ ]:
# Генерация списка карт и создание интерактивных виджетов

# Генерируем все карты с символами мастей
SUITS = {'s': '♠', 'h': '♥', 'd': '♦', 'c': '♣'}
RANKS = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']

# Создаём список всех карт в красивом формате
all_cards_display = []
all_cards_mapping = {}  # Отображение "A♠" -> Card.new('As')

for rank in RANKS:
    for suit_short, suit_symbol in SUITS.items():
        display_name = f"{rank}{suit_symbol}"
        card_code = f"{rank}{suit_short}"
        all_cards_display.append(display_name)
        all_cards_mapping[display_name] = Card.new(card_code)

# Добавляем опцию "Не выбрано" для карт стола
EMPTY_CARD = "-- (пусто)"

# Заголовок
header = widgets.HTML(
    value="<h3 style='color: #2e7d32; text-align: center;'>Выберите карты и параметры</h3>"
)

# Карты игрока
hero_label = widgets.HTML(value="<b>Карты игрока:</b>")
hero_card1 = widgets.Dropdown(
    options=all_cards_display,
    value='A♠',
    description='Рука:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='200px')
)
hero_card2 = widgets.Dropdown(
    options=all_cards_display,
    value='K♠',
    description='',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='130px')
)

# Карты стола (флоп, терн, ривер)
board_label = widgets.HTML(value="<b>Карты на столе:</b>")
board_card1 = widgets.Dropdown(
    options=[EMPTY_CARD] + all_cards_display,
    value=EMPTY_CARD,
    description='Флоп:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='200px')
)
board_card2 = widgets.Dropdown(
    options=[EMPTY_CARD] + all_cards_display,
    value=EMPTY_CARD,
    description='',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='130px')
)
board_card3 = widgets.Dropdown(
    options=[EMPTY_CARD] + all_cards_display,
    value=EMPTY_CARD,
    description='',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='130px')
)
board_card4 = widgets.Dropdown(
    options=[EMPTY_CARD] + all_cards_display,
    value=EMPTY_CARD,
    description='Терн:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='200px')
)
board_card5 = widgets.Dropdown(
    options=[EMPTY_CARD] + all_cards_display,
    value=EMPTY_CARD,
    description='Ривер',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='200px')
)

# Количество игроков
players_label = widgets.HTML(value="<b>Количество активных игроков:</b>")
players_slider = widgets.IntSlider(
    value=3,
    min=2,
    max=9,
    step=1,
    description='',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='400px')
)

# Количество симуляций
sims_label = widgets.HTML(value="<b>Количество симуляций:</b>")
sims_slider = widgets.IntSlider(
    value=10000,
    min=1000,
    max=100000,
    step=1000,
    description='',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='400px'),
    readout_format='d'
)

# Кнопка запуска
calculate_button = widgets.Button(
    description='🎲 Рассчитать',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px'),
    style={'font_weight': 'bold'}
)

# Область для вывода результатов
output_area = widgets.Output()


In [ ]:
# Обработка событий и визуализация результатов

def validate_cards() -> Tuple[bool, str]:
    """
    Проверка на дубликаты карт.

    Returns:
        (is_valid, error_message)
    """
    selected = []

    # Карты героя
    selected.append(hero_card1.value)
    selected.append(hero_card2.value)

    # Карты стола
    board_widgets = [board_card1, board_card2, board_card3, board_card4, board_card5]
    for widget in board_widgets:
        if widget.value != EMPTY_CARD:
            selected.append(widget.value)

    # Проверка на дубликаты
    if len(selected) != len(set(selected)):
        return False, "❌ Некоторые карты выбраны дважды!"

    return True, ""


def on_calculate_clicked(b):
    """
    Обработчик нажатия кнопки "Рассчитать".
    """
    with output_area:
        clear_output(wait=True)

        # Валидация
        is_valid, error_msg = validate_cards()
        if not is_valid:
            print(error_msg)
            return

        # Собираем карты героя
        hero_cards = [
            all_cards_mapping[hero_card1.value],
            all_cards_mapping[hero_card2.value]
        ]

        # Собираем карты стола
        board_cards = []
        board_widgets = [board_card1, board_card2, board_card3, board_card4, board_card5]
        for widget in board_widgets:
            if widget.value != EMPTY_CARD:
                board_cards.append(all_cards_mapping[widget.value])

        # Параметры
        active_players = players_slider.value
        n_sims = sims_slider.value

        # Информация о расчёте
        print(f"Запуск:        {n_sims:,} симуляций")
        print(f"Игроков:       {active_players}")
        print(f"Карты игрока:  {hero_card1.value} {hero_card2.value}")
        if board_cards:
            board_str = " ".join([widget.value for widget in board_widgets if widget.value != EMPTY_CARD])
            print(f"Карты стола:   {board_str}")
        else:
            print(f"Карты стола:   (Префлоп)")
        print("\n" + "="*50)

        # Расчёт
        wins, ties, losses = monte_carlo(hero_cards, board_cards, active_players, n_sims)


        # Вывод результатов
        print()
        print(f"Побед:        {wins:>6,}  ({wins/n_sims:.1%})")
        print(f"Ничьих:       {ties:>6,}  ({ties/n_sims:.1%})")
        print(f"Поражений:    {losses:>6,}  ({losses/n_sims:.1%})")
        print()

        # Столбчатая диаграмма
        colors = ['#4caf50', '#ffeb3b', '#f44336']
        fig, ax = plt.subplots(figsize=(9, 5))

        categories = ['Победы', 'Ничьи', 'Поражения']
        values = [wins, ties, losses]

        bars = ax.bar(categories, values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

        # добавим графику +15% места сверху
        ax.set_ylim(0, max(values) * 1.15)

        # Добавляем значения над столбцами
        for bar, val in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{val:,} ({val/n_sims:.1%})',
                    ha='center', va='bottom', fontsize=11, weight='bold')

        ax.set_ylabel('Количество симуляций', fontsize=12, weight='normal')
        ax.set_title(f'Результат на {n_sims:,} симуляций', fontsize=14, weight='bold')
        ax.grid(axis='y', alpha=0.3, linestyle='--')

        plt.tight_layout()
        plt.show()


# Привязываем обработчик к кнопке
calculate_button.on_click(on_calculate_clicked)

In [ ]:
# Отображение интерфейса

# Компоновка виджетов
interface = widgets.VBox([
    header,
    widgets.HTML(value="<hr>"),
    hero_label,
    widgets.HBox([hero_card1, hero_card2]),
    widgets.HTML(value="<br>"),
    board_label,
    widgets.VBox([
        widgets.HBox([board_card1, board_card2, board_card3]),
        widgets.HBox([board_card4]),
        widgets.HBox([board_card5])
    ]),
    widgets.HTML(value="<br>"),
    players_label,
    players_slider,
    widgets.HTML(value="<br>"),
    sims_label,
    sims_slider,
    widgets.HTML(value="<br><hr>"),
    widgets.HBox([calculate_button], layout=widgets.Layout(justify_content='center')),
    output_area
], layout=widgets.Layout(padding='20px', border='2px solid #ddd', border_radius='10px'))

# Отображаем интерфейс
display(interface)